Import libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import geopandas as gpd
import copy as cp
import json

In [2]:
# make functions to celan census
def clean_census_data(data):
    # remove unwanted columns
    clean_data = (data.drop(['CDU_ID', 'GEO_LABEL', 'GEO_TYPE', 'GEO_TYP2'], axis =1)\
                  .drop([0], axis = 0)\
                  .set_index('GEO_CODE')\
                  .T)
    clean_data = clean_data.dropna(axis = 0).T
    return(clean_data)
    
def get_meta_data(data):
    # remove unwanted columns
    meta_data = (data.dropna(axis = 1).iloc[:1].T)
    # separate into various columns
    for i in range(meta_data.iloc[0][0].count(' - ')):
        name = str(meta_data.iloc[0][0].split(' - ')[i].split(' : ')[0])
        # extract individual strings
        temp_list = []
        for j in range(len(meta_data)):
            var_string = str(meta_data.iloc[j][0].split(' - ')[i].split(' : ')[1])
            temp_list.append(var_string)
            #print(temp_list)
        # make columns
        meta_data[name] = temp_list
        #print(temp_list)
    # remove unwanted column
    meta_data = meta_data.drop([0], axis = 1)
    # clean headers
    meta_data.columns = meta_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(')', '').str.replace('(', '').str.replace(';', '')
    return(meta_data)
    

In [3]:
# files

cen_age = pd.read_csv('Data/Census/DATA_AGE_UNIT.csv')
cen_bdr = pd.read_csv('Data/Census/DATA_BEDR_UNIT.csv')
cen_dpr = pd.read_csv('Data/Census/DATA_DEPR_UNIT.csv')
cen_eth = pd.read_csv('Data/Census/DATA_ETH_UNIT.csv')
cen_hth = pd.read_csv('Data/Census/DATA_HEALTH_UNIT.csv')
cen_sex = pd.read_csv('Data/Census/DATA_SEX_UNIT.csv')
cen_hhd = pd.read_csv('Data/Census/DATA_HHDCOM_UNIT.csv')
cen_ecn = pd.read_csv('Data/Census/DATA_ECON_UNIT.csv')
cen_hsz = pd.read_csv('Data/Census/DATA_HSZ_UNIT.csv')
cen_buk = pd.read_csv('Data/Census/DATA_BUK_UNIT.csv')
cen_rno = pd.read_csv('Data/Census/DATA_RNO_UNIT.csv')
cen_sec = pd.read_csv('Data/Census/DATA_NSSeC_UNIT.csv')
cen_rlg = pd.read_csv('Data/Census/DATA_RELIG_UNIT.csv')
cen_cob = pd.read_csv('Data/Census/DATA_COB_UNIT.csv')
cen_mrg = pd.read_csv('Data/Census/Data_AGESST_UNIT.csv')

In [4]:
# make lists and run throguh function

census_list = [cen_age, cen_bdr, cen_dpr, cen_eth, cen_hth, cen_sex, cen_hhd, cen_ecn, cen_hsz, cen_buk, cen_rno, cen_sec, cen_rlg, cen_cob, cen_mrg]
census_str = ['age', 'bdr', 'dpr', 'eth', 'hth', 'sex', 'hhd', 'ecn', 'hsz', 'buk', 'rno', 'sec', 'rlg', 'cob', 'mrg']

In [5]:
# clean and save to file
meta_list = []
data_list = []
for i in range(len(census_list)):
    # clean df
    cen_df = clean_census_data(census_list[i])
    data_list.append(str('Data/Census/' + census_str[i] + '_data.csv'))
    cen_df.to_csv(data_list[i])
    
    # meta data
    meta_df = get_meta_data(census_list[i])
    meta_list.append(str('Data/Census/' + census_str[i] + '_meta.csv'))
    meta_df.to_csv(meta_list[i])

In [6]:
# get fractions and merge dataframes
all_data = pd.read_csv(data_list[0]).set_index('GEO_CODE')

for i in range(1, len(data_list)):
    df = pd.read_csv(data_list[i]).set_index('GEO_CODE')
    df['total'] = df.sum(axis = 1)
    for j in range(len(df.columns)):
        df.iloc[:, j] = df.iloc[:, j] / df['total']
    all_data = all_data.join(df.drop('total', axis =1))

all_data.to_csv('Data/Census/all_data.csv')

In [7]:
# make dictionary
meta_data = {}
for i in range(len(meta_list)):
    entry = pd.read_csv(meta_list[i]).set_index('Unnamed: 0').iloc[:, 0].to_dict()
    meta_data.update(entry)

In [8]:
json = json.dumps(meta_data)
f = open("Data/Census/meta_dict.json","w")
f.write(json)
f.close()